In [1]:
# import the wrapper
from mcmc import *
from power_spectra_1D import *
from power_spectra_2D import *

[potato:85466] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.potato.1000/jf.0/3962765312/shared_mem_cuda_pool.potato could be created.
[potato:85466] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 


In [ ]:
sim = Simulation(save_ondisk=False, save_inclass=True, write_cache=True, clean_cache=True)
# }
#{'ION_Tvir_MIN': 4.540949748007259, 'HII_EFF_FACTOR': 238.77818673489438, 'L_X': 39.63512786298546, 'NU_X_THRESH': 1156.8820204520493}
#params={'astro_params': {'ION_Tvir_MIN': 4.798687324224536, 'HII_EFF_FACTOR': 153.93554078107047, 'L_X': 41.07914816577325, 'NU_X_THRESH': 122.31064697905447}}
for mfp in [0]: #np.linspace(1,50, 5):
    params = {'astro_params': {'ION_Tvir_MIN': 4.7, 'HII_EFF_FACTOR': 30, 'L_X': 40, 'NU_X_THRESH': 500}}
    #{'astro_params': {'ION_Tvir_MIN': 4.7, 'HII_EFF_FACTOR': 30, 'L_X': 40, 'NU_X_THRESH': 500}}
    sim.run_lightcone(kargs=params)

'''

# w/o write: 
# w write: 
# 11.12.2s

user_params = p21c.UserParams(
    HII_DIM=60, BOX_LEN=200
)
flag_options = p21c.FlagOptions(
    INHOMO_RECO=True, USE_TS_FLUCT=True
)

astro_params = p21c.AstroParams(
    **{'ION_Tvir_MIN': 4.7, 'HII_EFF_FACTOR': 30, 'L_X': 40, 'NU_X_THRESH': 500}
)

lightcone = p21c.run_lightcone(
    direc='_cache',
    user_params=user_params,
    flag_options=flag_options,
    astro_params=astro_params,
    redshift = 5.5
)
'''


In [ ]:
#(array([ 9,  9,  9, 44, 44, 44]),
#array([68, 68, 68, 14, 14, 14]),
#array([ 62, 182, 302,  68, 188, 308]))

'''
(array([ 63,  63,  63, 104, 104]),
 array([61, 61, 61, 66, 66]),
 array([ 80, 200, 320, 105, 225]))
 '''

def compute_ps(data, k_bins=31, z_bins=21):
    zbins = np.linspace(0,data.lightcone_redshifts.shape[0]-1,z_bins).astype(int)
    ps = np.empty((z_bins-1, 2, k_bins))
    for bin in range(z_bins - 1):
        physical_size = data.lightcone_distances[zbins[bin+1]] - data.lightcone_distances[zbins[bin]]
        # get variance=False for now until nice usecase is found
        ps[bin,:,:] = get_power(deltax= data.brightness_temp[:,:,zbins[bin]:zbins[bin+1]], 
        boxlength=(*data.lightcone_dimensions[:2], physical_size), bin_ave=True, 
        ignore_zero_mode=True, get_variance=False, bins=k_bins, vol_normalised_power=True)
        ps[bin,0,:] *= ps[bin,1,:]**3/(2* np.pi**2)
        print(f"{bin=}: ", f"{ps[bin,:,:]}")
    return ps
compute_ps(sim.data[0])


In [ ]:
from mcmc import *

init_params_ranges = {"astro_params": {"ION_Tvir_MIN": [4,5.3], "HII_EFF_FACTOR": [10, 250], 
                    "L_X": [38,42], "NU_X_THRESH": [100, 1500]}}

params={'astro_params': {'ION_Tvir_MIN': 4.7, 'HII_EFF_FACTOR': 30, 'L_X': 40, 'NU_X_THRESH': 500}}

def log_prior(theta):
        T_vir, H_eff, LX, nu_x = theta
        if 5 < H_eff < 250 and 100 < nu_x < 1500 and 4 < T_vir < 5.3 and 30 < LX < 50:
            return 0
        return - np.inf
    
def log_likelihood(t,f):
    return - np.sum((t - f)**2/(f+1))

mcrun = mcmc(nwalker=8, z_bins = 21, k_bins=30, debug = True, log_likelihood=log_likelihood, prior=log_prior)
mcrun.make_fiducial(load = False, fparams=params, plot=True)

#mcrun.run_aies(init_params_ranges)

In [ ]:
from p21cmfastwrapper import *

sim = Simulation(save_ondisk=True, save_inclass=False, write_cache=True, clean_cache=True)

args = {"M_WDM":[0.3,10.0],
                "astro_params": {"HII_EFF_FACTOR":[10,250],"L_X":[38,42],"NU_X_THRESH":[100,1500],"ION_Tvir_MIN":[4.0,5.3]},
                "cosmo_params": {"OMm":[0.2,0.4]}}
sim.run_samplef(nruns=50000, args=args=args, threads = 128)

In [ ]:
def fill_dict(nested_dict, array, index=0):
    """
    Fills the nested dictionary with elements from the array recursively.

    Parameters:
    nested_dict (dict): The nested dictionary to be filled.
    array (list): The list of elements to fill into the nested dictionary.
    index (int): The current index in the array.

    Returns:
    int: The next index to be used in the array.
    """
    for key in nested_dict:
        if isinstance(nested_dict[key], dict):
            index = fill_dict(nested_dict[key], array, index)
        else:
            if index < len(array):
                nested_dict[key] = array[index]
                index += 1
            else:
                break
    return nested_dict

In [ ]:
log_prior([  4.01547017, 245.68181515,  41.78008556, 142.73262024]) + log_likelihood(ps, mcrun.fid_ps) 

In [ ]:
from mcmc import *

init_params_ranges = {"astro_params": {"ION_Tvir_MIN": [4,5.3], "HII_EFF_FACTOR": [10, 250], 
                    "L_X": [38,42], "NU_X_THRESH": [100, 1500]}}

params={'astro_params': {'ION_Tvir_MIN': 4.7, 'HII_EFF_FACTOR': 30, 'L_X': 40, 'NU_X_THRESH': 500}}

def log_prior(theta):
        T_vir, H_eff, LX, nu_x = theta
        if 5 < H_eff < 250 and 100 < nu_x < 1500 and 4 < T_vir < 5.3 and 30 < LX < 50:
            return 0
        return - np.inf
    
def log_likelihood(t,f):
    return - np.sum((t - f)**2/(np.abs(f)+1))

mcrun = mcmc(nwalker=8, z_bins = 11, k_bins=30, debug = True, log_likelihood=log_likelihood, prior=log_prior)
mcrun.make_fiducial(load = True, fparams=params)

#mcrun.run_aies(init_params_ranges)

In [ ]:
from mcmc import *
init_params_ranges = {"astro_params": {"ION_Tvir_MIN": [4,5.3], "HII_EFF_FACTOR": [5, 100], 
                    "L_X": [30,50], "NU_X_THRESH": [100, 1500]}}

# input: init cube for parameter
def prior_sampling(theta):
        T_vir, H_eff, LX, nu_x = theta
        T_vir = 4 + 1.3*T_vir
        H_eff = 5 + 95*H_eff
        LX = 30 + 20*LX
        nu_x = 100 + 1400*nu_x
        return np.array([T_vir, H_eff, LX, nu_x])
    
# returns samples from the posterior
    
def log_likelihood(t,f):
    return - np.sum((t - f)**2/f)


mcrun = mcmc(z_bins = 11, debug = True, log_likelihood=log_likelihood, prior=prior_sampling)
mcrun.make_fiducial(load = True)

mcrun.run_ns(init_params_ranges)


In [ ]:
from mcmc import *
mcrun = mcmc(nwalker=8, z_bins = 11, debug = True, log_likelihood=log_likelihood, prior=log_prior)
mcrun.make_fiducial(load = False)

In [ ]:
import py21cmfast
py21cmfast.inputs.CosmoParams._defaults_

In [ ]:
args = {"astro_params": 
        {"NU_X_THRESH": [400,700], 
        "HII_EFF_FACTOR": [10,100],
        "L_X": [1,2],
        "ION_Tvir_MIN": [4,5]},
    "cosmo_params":
        {"OMm": [0,1]},
    "global_params": 
        {"M_WDM": [1,2]}}

In [ ]:
py21cmfast.inputs.global_params.M_WDM

In [ ]:
test = {"main": {"blub": 1, "blubber": 2}, "sek": {"schwubble": {"swap": 4}}}

In [ ]:
def extract_keys(nested_dict):
    keys = []
    for key in nested_dict:
        keys.append(key)
        if isinstance(nested_dict[key], dict):
            keys.extend(extract_keys(nested_dict[key]))
    return keys

In [ ]:
extract_keys(test)